In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from Meanfield_Dyn_util import *


In [2]:
### dynamics
def iidGaussian(stats,shapem):
	mu,sig = stats[0],stats[1]
	nx,ny = shapem[0],shapem[1]
	return np.random.normal(mu,sig,(nx,ny))

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`
    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.
    Returns
    -------
    matplotlib.patches.Ellipse
    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d
def equations(x,data):
    '''
    Function: compute the largest real eigenvalue outlier of the matrix with chain motifs
    eqleft = (outlier**2+gaverage**2*tau)*(outlier**2-gaverage**2*tau*(N-1))
    eqright = outlier*eigvAm[0]*(outlier**2-gaverage**2*tau*(N-1))+gaverage**2*tau*eigvAm[0]*N
    f(outlier) = eqleft - eqright = 0
    params:
    data = [g, tau_chn, lambda_0, N]
    '''
    outlier = x
    gaverage,tau,eigvAm,N = data[0],data[1],data[2],data[3]
    eqleft = (outlier**2+gaverage**2*tau)*(outlier**2-gaverage**2*tau*(N-1))
    eqright = outlier*eigvAm*(outlier**2-gaverage**2*tau*(N-1))+gaverage**2*tau*eigvAm*outlier*N
    # eqright = outlier*eigvAm*(outlier**2+gaverage**2*tau)
    
    return (eqleft-eqright)
#### compute corresponding parameters and variables in the equivalent sparse networks
### Epopulation
def compute_AdjacencyParams(x,JE,JI,sigmae,sigmai,Nparams):
    ce,ci = x[0],x[1]
    NE,NI = Nparams[0],Nparams[1]
    J = JE/NE/ce 
    g = JI/NI/ci/J 
    
    sigmae2 = J**2*ce*(1-ce)*N 
    sigmai2 = g**2*J**2*ci*(1-ci)*N 
    
    resultse = sigmae**2-sigmae2
    resultsi = sigmai**2-sigmai2
    return np.array([resultse,resultsi])

### Network with Gaussian-distributed synaptic weights

In [3]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@(x)+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)

In [4]:
nn = [50,100,150,250,750,500,800]
NE, NI = nn[-1], nn[-1] 
gamma = 1/4
NI = int(NE*gamma)
N = NE+NI
JE, JI = 0.65, 1.65#1.5, 2.0
Nparams = np.array([NE,NI])
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 

tt  = np.linspace(0,100,1000)
g_max,tau_max = 0.10,.1
ntau = 6### even quit sparse taus can generate accurate predictions
ntau = 11## denser taus
tau_series = np.linspace(0,tau_max,ntau)#np.linspace(0,0.2,ntau)
trials = 30+6 #is the minimal number of iterations
# recordings
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
htau    = tau_series[1]-tau_series[0]

### also record the reconstructed eigenvectors
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))

### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0] = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left

In [5]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert[:NE]=0 ### inhibitory input 
# Ipert[NE:]=0 ### excitatory input
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
tt    = np.linspace(0,100,1000)
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [6]:
### if you want to re-run the simulations
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
firing_ratepert = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))

intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population

gaverage = g_max ### set the magnitude of the connectivity fluctuation
switch = 1

norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

for ktrial in range(trials):
    xr      = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
    chneta  = iidGaussian([0,gaverage/np.sqrt(N)],[N,6])
    xrec    = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
    ### zscore
    xr   = stats.zscore(xr.flatten())
    xr   = xr*gaverage/np.sqrt(N)
    xr   = np.reshape(xr,(N,N))
    ### zscore
    xrec = stats.zscore(xrec.flatten())
    xrec = xrec*gaverage/np.sqrt(N)
    xrec = np.reshape(xrec,(N,N))
    ### zscore
    chneta[:,0]  = stats.zscore(chneta[:,0])
    chneta[:,0] *=(gaverage/np.sqrt(N))
    ### ---------------------
    intg_ov = np.zeros(2)
    intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
    mean_pre = np.array([JE-JI,0])
    mean_total_change = np.zeros(2)
    z_pre = np.zeros((N,N))
    for it, tau in enumerate(tau_series):
        a    = np.sqrt(np.abs(tau))
        zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
        zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)#a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
        gammarec = a*xrec-a*xrec.copy().T
        # zr   = zrow.copy()+zcol.copy()+np.sqrt(1-4*tau)*xr+gammarec
        zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr
        # zr = xrec*np.sqrt(tau/2)+xrec.copy().T*np.sqrt(tau/2)+np.sqrt(1-tau)*xr ### RECIPROCAL        
        ha    = np.sqrt(htau)
        hzrow = ha*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
        hzcol = ha*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
        hgammarec = ha*xrec-ha*xrec.copy().T
        # hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-4*htau)*xr+hgammarec
        hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-2*htau)*xr
        DELTA_Z = zr-z_pre
        hzr   = DELTA_Z.copy()
        
        ### generate J connectivity matrix
        Jchn = Jbar.copy()+zr.copy()
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jchn, Inp)
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
        ### perturbation 
        xpert = xc_temporal[-1,:].copy()
        xpert = xpert.reshape(-1,1)
        dtt =tt[1]-tt[0]
        for ttt in range(len(tt)):
            delta_x= -xpert + Jchn@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
            xpert = delta_x*dtt+xpert 
        firing_ratepert[ktrial,it,:] = xpert.copy().squeeze()
        
        eigvchn, eigrvec = la.eig(Jchn)
        eigvchn_,eiglvec = la.eig(Jchn.copy().T)
        assert(np.sum(np.abs(eigvchn[0]-eigvchn_[0]))<1e-9)## check the consistency of left and right eigenvectors
        ### normalization
        reig  = np.squeeze(eigrvec[:,:].copy())
        leig0 = np.squeeze(eiglvec[:,:].copy()) 
        normval = np.sum(reig.copy()*leig0.copy(),axis=0)
        norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
        normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        leig = leig0.copy()/normval.copy()### left eigenvector normalization

        if np.mean(reig[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
            
        ### numerical low-rank approximation 
        ov_inp_lowrank,ov_inp_lowrank_div= np.zeros(2),np.zeros(2)
        vec_lowrank_contribution = np.zeros((N,2))
        ### linear response theory approximation 
        for i in range(2):
            ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
            ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
        Equilibrium_lowrank_outliers = Inp.copy()
        for i in range(2):
            vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
            Equilibrium_lowrank_outliers += vec_lowrank_contribution[:,i].copy()
            #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
            contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
            contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
        lowrank_eq_num[ktrial,it,:] = Equilibrium_lowrank_outliers.copy()
        ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
        
        eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
        for iii in range(2):
            eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
        # chain_mtx = np.ones((N,N))-np.eye(N)
        # chain_mtx = gaverage**2*chain_mtx
        chain_mtx = hzr@hzr/htau
        chainE_mtx = gaverage**2*(np.ones((N,N))-np.eye(N))*NE/N #### ERROR!!!! (CHAIN_MTX)xxxx
        chainI_mtx = gaverage**2*(np.ones((N,N))-np.eye(N))*NI/N
        if it<1:
            lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
            # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
            eigeng     = np.array([JE-JI,0])
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
        else:
            lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
            eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
            ### conditioned mean  
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
            rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
            
        ### get the appropriate normalization factor
        norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
        if it==0:
            hzr_u = xr.copy()
            with_chn = 0
        else:
            hzr_u = hzr.copy()
            with_chn = 1
        
        norm_rvec_temp = np.zeros((N,2))
        norm_lvec_temp = np.zeros((N,2))
        for i in range(2):
            rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
            lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
            
            '''MEAN CONNECTIVITY'''
            current_eigv = eigvchn_series[ktrial,it,i].copy()
            if it==0:
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            elif it < 3 and i==1:#1:
                ### original values are obtained numerically
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])             
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            else:     
                eigenvalue_u = np.real(current_eigv)
                eigenvalue_um = np.real(eigeng[i]) 
                
                ### norm_for_rvec and norm_for_lvec 
                rmean_tmp = np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*htau*(chain_mtx@np.reshape(rvec_mean[:,i],(-1,1)))/eigenvalue_um**2,(N,1)) ### mean-shifting 
                rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]

                lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*htau*np.reshape(lvec_mean[:,i],(1,-1))@chain_mtx/np.real(eigeng[i])**2,(N,1))### mean-shifting 
                lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                
                norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                
                norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))       
                norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                ### need to be re-normalized
                norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
                norm_lvec_temp[:,i] *= eigvchn_series[ktrial,it,i].real
                
                intg_ml[:,i] = lmean_tmp[:,0]/norm_for_lvec[i]*eigenvalue_u
                leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norm_for_lvec[i]
                leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*htau*np.reshape(lvec_mean[:,i],(1,-1))@chain_mtx/eigeng[i].real**2,(N,1))[:,0]#*eigenvalue_u#    
                intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                
                tilden = np.reshape(lvec_n.copy(),(-1,1))/norml0_series[ktrial,it,i]
                norm_4lvec_series_[ktrial,it,i]=np.reshape(tilden,(1,-1))@np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i]
                
                norm_4rvec_series[ktrial,it,i],norm_4lvec_series[ktrial,it,i]=norm_for_rvec[i],norm_for_lvec[i]
                if i==0:
                    print(i,'should be the same',norm_4lvec_series[ktrial,it,i]/norml0_series[ktrial,it,i],norm_4lvec_series_[ktrial,it,i])
            
            norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
            intg_std_series[ktrial,it,i,0] = np.std(norm_rvec_temp[:NE,i].copy())
            norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
            intg_std_series[ktrial,it,i,1] = np.std(norm_rvec_temp[NE:,i].copy())
            norm_rvec_temp[:NE,i] += intg_mr[:NE,i]
            norm_rvec_temp[NE:,i] += intg_mr[NE:,i]         
            ### and then the norm_lvec_temp
            norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
            norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
            norm_lvec_temp[:NE,i] += intg_ml[:NE,i]
            norm_lvec_temp[NE:,i] += intg_ml[NE:,i]
            ### also record the reconstructed eigenvectors
            eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
            eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
            
            intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
            intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
            
            ''' OVERALL OVERLAP '''
            ### after correction (step by step)
            if it>0:
                if it<=switch:
                    eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
                else:
                    eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
                intg_ov[i] = first_perturb_ov[ktrial,it-1,i,i] + with_chn*htau*lvec_mean[:,i].T@chain_mtx@rvec_mean[:,i]/np.real(eigeng[i])/np.real(eigenvalues_usem[i])/norm_for_lvec[i]/norm_for_rvec[i]*np.real(eigenvalues_use[i])
            else:
                intg_ov[i] = lvec_mean[:,i].T@(hzr_u@hzr_u)@rvec_mean[:,i]/np.real(eigeng[i])**2/norm_for_lvec[i]/norm_for_rvec[i]*eigvchn_series[ktrial,it,i].real
            
            intg_ov_series[ktrial,it,i] = np.squeeze(intg_ov[i]).real
        
        z_pre = zr.copy()
        ### theoretically compute the equilibrium population-averaged firing rate
        an = np.zeros((2,2),dtype=complex)
        am = np.zeros((2,2),dtype=complex) ## population X rank    
        for ir in range(2):
            an[0,ir] = np.mean(intg_ml[:NE,ir])
            an[1,ir] = np.mean(intg_ml[NE:,ir])
            am[0,ir] = np.mean(intg_mr[:NE,ir])
            am[1,ir] = np.mean(intg_mr[NE:,ir])
            
        ### overlap sum
        overlap_inp = np.zeros(2,dtype=complex) 
        for ir in range(2):
            overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
            ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
            overlap_inp[ir]/= (1.0-eigvchn[ir])
            ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
        eq_fr = np.zeros(2)
        eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
        for ir in range(2): ### rank two 
            ### record 
            contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
            contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
            eq_fr[0] += am[0,ir]*overlap_inp[ir]
            eq_fr[1] += am[1,ir]*overlap_inp[ir]
        lowrank_eq[ktrial,it,:] = eq_fr.copy() 
         
        '''overlap per population'''
        if it>=1:
            if it<=switch:
                eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
            else:
                eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
            intg_ovP = first_perturb_ovP[ktrial,it-1,:,:,:].copy()
            for ir in range(2):
                for jl in range(2):
                    intg_ovP[ir,jl,0] += with_chn*htau*lvec_mean[:,jl].T@(chainE_mtx)@rvec_mean[:,ir]/norm_for_lvec[jl]/norm_for_rvec[ir]*eigenvalues_use[jl].real/np.real(eigeng[jl])/np.real(eigenvalues_usem[ir])### current eigenvalues are multiplied
                    intg_ovP[ir,jl,1] += with_chn*htau*lvec_mean[:,jl].T@(chainI_mtx)@rvec_mean[:,ir]/norm_for_lvec[jl]/norm_for_rvec[ir]*eigenvalues_use[jl].real/np.real(eigeng[jl])/np.real(eigenvalues_usem[ir])### current eigenvalues are multiplied                  
                    intg_crossovPop_series[ktrial,it,ir,jl,0] = np.squeeze(intg_ovP[ir,jl,0]).real
                    intg_crossovPop_series[ktrial,it,ir,jl,1] = np.squeeze(intg_ovP[ir,jl,1]).real
        
        ### mean shift                    
        meanr, meanl = np.zeros((2,2)),np.zeros((2,2))## rank pop
        meanr[:,0], meanr[:,1] = np.mean(reig[:NE,:2],axis=0),np.mean(reig[NE:,:2],axis=0)
        meanl[:,0], meanl[:,1] = np.mean(leig[:NE,:2],axis=0),np.mean(leig[NE:,:2],axis=0)
        
        ##### compute current overlap (based on current mean connectivity)
        noiser, noisel = np.zeros((N,2)),np.zeros((N,2))
        for i in range(2):
            noiser[:NE,i], noiser[NE:,i] = reig[:NE,i] - meanr[i,0], reig[NE:,i]-meanr[i,1]
            noisel[:NE,i], noisel[NE:,i] = leig[:NE,i] - meanl[i,0], leig[NE:,i]-meanl[i,1]
        # noiser[:NE], noiser[NE:] = reig[:NE,0] - meanr[0], reig[NE:,0]-meanr[1]
        # noisel[:NE], noisel[NE:] = leig[:NE,0] - meanl[0], leig[NE:,0]-meanl[1]
        for ir in range(2):
            for jl in range(2):
                first_perturb_ov[ktrial,it,ir,jl] = np.squeeze(np.reshape(noiser[:,ir],(1,-1))@np.reshape(noisel[:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])#@YS 17Nov change to fit ovP#*np.real(eigeng[jl]) #
                ### separate E and I population
                first_perturb_ovP[ktrial,it,ir,jl,0]= np.squeeze(np.reshape(noiser[:NE,ir],(1,-1))@np.reshape(noisel[:NE,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])
                first_perturb_ovP[ktrial,it,ir,jl,1]= np.squeeze(np.reshape(noiser[NE:,ir],(1,-1))@np.reshape(noisel[NE:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl]) ### current eigenvalues are multiplied
                


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:93: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:111: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:115: ComplexWarning: Casting complex values to real discards the imaginary part
  vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:124: ComplexWarning: Casting complex values to real dis

0 should be the same 1.1572912929473396 1.1572912929473391


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:172: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:173: ComplexWarning: Casting complex values to real discards the imaginary part
  intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:174: ComplexWarning: Casting complex values to real discards the imaginary part
  leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:175: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9188\2897460650.py:176: ComplexWarning: Cas

0 should be the same 1.0461776730104582 1.046177673010458
0 should be the same 1.033111232111844 1.033111232111844
0 should be the same 1.0266378447499236 1.0266378447499236
0 should be the same 1.0225803961450985 1.0225803961450985
0 should be the same 1.0197362943812025 1.0197362943812023
0 should be the same 1.0176057502020217 1.0176057502020215
0 should be the same 1.0159372757012812 1.0159372757012812
0 should be the same 1.0145882351302586 1.0145882351302586
0 should be the same 1.013470764776104 1.0134707647761043
0 should be the same 1.1586642875926778 1.1586642875926778
0 should be the same 1.0538113216497322 1.053811321649732
0 should be the same 1.0392077495945211 1.0392077495945211
0 should be the same 1.031634593027783 1.031634593027783
0 should be the same 1.026784069566567 1.026784069566567
0 should be the same 1.0233467513919412 1.0233467513919414
0 should be the same 1.0207578440161176 1.0207578440161176
0 should be the same 1.0187256424964208 1.0187256424964208
0 shou

In [21]:
# data_name ="E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Gaussian_data/DynsStats_Gauss_16April_EXParadoxical_normlr.npz"# "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Gaussian_data/DynsStats_Gauss_16April_INParadoxical_normlr.npz"
# data_name = "your data folder/DynsStats_Gauss_12Feb_INParadoxical.npz"
# np.savez(data_name, **data)

data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Gaussian_data/DynsStats_Gauss_16April_EXParadoxical_normlr.npz"#DynsStats_Gauss_16April_INParadoxical_normlr.npz"#
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']

firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

In [22]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx#[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [23  6  1 31 24 13 16  5 32 34 33  8 14  2 28 21 22 35 15  4  9 19 20 27
  0  7 10 30 26 11 25 12 29 17  3 18] 36


In [23]:
### compute the theoretical eigenvalue outliers and the spectral radius
### recording 
gaverage = g_max
eigvchn_theo = np.zeros((ntau,2))
radius_theo_map  = np.zeros(ntau)
        
for it, tau in enumerate(tau_series):
    ### solve the equations
    sigmae, sigmai = gaverage, gaverage
    
    x0 = np.array([0.1,0.1])
    sol = fsolve(compute_AdjacencyParams,x0,args=(JE,JI,sigmae,sigmai,Nparams))
    ce,ci = sol[0],sol[1]
    J = JE/NE/ce
    g = JI/NI/ci/J
    # print('ce,ci:',ce,ci)
    # print('J,g:',J,g)
    ### generate the corresponding sparse matrix
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    tau_rec_vec = np.array([[2*tau,-2*tau],[-2*tau,2*tau]])
    # tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,-tau],[-tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
    ### then the theoretical solutions for the eigenvalue outliers
    ### theoretical solutions
    x0 = np.mean(eigvchn_series[:,it,:2].real,axis=0)
    N = int(NE+NI)
    for ir in range(2):
        t1 = fsolve(equations,x0[ir].real,args=[gaverage,tau,(JE-JI),N])
        eigvchn_theo[it,ir] = t1

/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_6865/4030619877.py:61: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigvchn_theo[it,ir] = t1


In [59]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [37]:
### plot a shifting histogram for the first eigenvalue
fig,ax=plt.subplots(figsize=(6,3))
ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
### set double ylabels
ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
ax2 = ax.twinx() 
 
color = 'tab:gray'
ax2.set_ylabel('probability density\n (numerical)', color = color,fontsize=14)
ax2.tick_params(axis ='y', labelcolor = color) 
ax2.set_yticks([])
ax2.spines['bottom'].set_color('none')
### set other x-axis invisible
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['left'].set_color('none')
# ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
### refine the plot 
# ax.set_xlim([-3,2])
# ax.set_ylim([-0.3,0.3])
ax.set_xlim([-0.3,0.3])
ax.set_ylim([-0.3,0.3])
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set the x and y ticks
# ax.set_xticks([-3,0,2])
# ax.set_yticks([-0.3,0,0.3])


# ### also, plot the theoretical predictions for the first and second eigenvalues
### plot a circle with the radius of the last radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[-1],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
### plot a circle with the radius of the first radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

ktrial_index = 16
### scatter plot the eigenvalue bulk of this trial
### randomly select 500 points
idx_random = np.random.choice(np.arange(2,N),size=500,replace=False)
ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=5,alpha=0.2)
### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
ax.scatter(JE-JI,0,color='red',marker='X',s=80,alpha=1)


In [38]:
### scatter plot the predicted eigenvalues eigvchn_theo against the numerical mean np.mean(eigvchn_series[:,:,0],axis=0)
fig, ax = plt.subplots(figsize=(4,4))
### theoretical eigenvlaues eigvchn_theo against the numerical mean np.mean(eigvchn_series,axis=0)
ax.scatter(np.mean(eigvchn_series[idx_eff,:,0],axis=0),eigvchn_theo[:,0],cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
ax.scatter(np.mean(eigvchn_series[idx_eff,:,1],axis=0),eigvchn_theo[:,1],cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')
### refine the figure
ax.set_xticks([-3,0,3])
ax.set_yticks([-3,0,3])
ax.set_xlim([-3,3])
ax.set_ylim([-3,3])
### set x-axis and y-axis to zero
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
ax.plot([-3,3],[-3,3],lw=1.5,color='gray',linestyle='--',alpha=0.5)
ax.set_xlabel('mean of numerical')
ax.set_ylabel('theoretical prediction')

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\collections.py:192: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Text(0, 0.5, 'theoretical prediction')

In [24]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
thl = 10
ths = 0.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        

In [61]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
# ax[0].set_ylim(-0.01,0.06)
# ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-10,10)
# ax[0].set_yticks([0,0.05])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-0.4,0.02)
# ax[1].set_yticks([-0.4,0.])
# ax[1].set_ylim(-0.35,0.02)
# ax[1].set_yticks([-0.35,0.])
# ax[1].set_ylim(-10,10)
# ax[1].set_yticks([0,0.05])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [62]:
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=1.0)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
ax[0].set_ylim(-0.01,0.04)
ax[0].set_yticks([0,0.04])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)

### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
ax[1].set_ylim(-0.01,0.04)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

In [15]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir].real
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))

### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
for it in range(ntau):
    response_func[:,it,1] = 1
    if it>=0:#=1:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
    # elif it==1:
    #     for ir in range(2):
    #         if ir==0:
    #             response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    #             response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    #         else:
    #             response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-radius_theo_map[it].real)
    #             response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-radius_theo_map[it].real)
                
    else:
        for ir in range(1):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)



In [15]:
### compute mean overlap 
pureE = np.zeros((trials,ntau)) 
for ktrial in range(trials):
    for it in range(ntau):
        pureE[ktrial,it] = np.sum(eiglvec_series[ktrial,it,:NE,0]*eigrvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0])
        pureE[ktrial,it] += np.sum(eiglvec_series[ktrial,it,:NE,1]*eigrvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1])

/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_22740/2064652928.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  pureE[ktrial,it] = np.sum(eiglvec_series[ktrial,it,:NE,0]*eigrvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0])
/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_22740/2064652928.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  pureE[ktrial,it] += np.sum(eiglvec_series[ktrial,it,:NE,1]*eigrvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1])


In [16]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
# #### delete the largest 3 and the smallest 3 from response_func_num
# for it in range(ntau):
#     idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>1e2)
#     idxnan = (idxsort) 
#     response_func_num[idxnan,it,0] = np.nan 
    
    
#     idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>1e2)
#     idxnan = (idxsort) 
#     response_func_num[idxnan,it,1] = np.nan 

In [19]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')
ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)

# ax.plot(tau_series,response_func_contribution[:,0,0]+response_func_contribution[:,1,0],'green',marker='o')
# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([-2,2])
ax.set_yticks([-2,2])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])

plt.show()

In [20]:

### compare response_func with response7 _func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.mean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,1],'k',marker='o')
# ax.plot(tau_series,np.mean(pureE,axis=0),'r',marker='o')

ax.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)

alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('inhibitory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([-0.5,1.5])

ax.set_yticks([-1.,0.2])
ax.set_ylim([-1.,0.25])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])

plt.show()

2024-06-28 20:16:30.501 python[6865:254516] Warning: Window move completed without beginning
2024-06-28 20:17:01.284 python[6865:254516] Warning: Window move completed without beginning
2024-06-28 20:17:15.770 python[6865:254516] Warning: Window move completed without beginning


In [56]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(5,3))

ax.plot(tau_series,eigvchn_theo[:,0],'tab:orange',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
ax.set_ylim([-4.5,3.5])
ax.set_yticks([-4,0,3])

ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
### plot y==1 
ax.plot([0,tau_series[-1]],[1,1],color='gray',linestyle='--',linewidth=1.5)
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

#### excitatory

In [29]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
# Ipert[:NE]=0 ### inhibitory input 
Ipert[NE:]=0 ### excitatory input
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
tt    = np.linspace(0,100,1000)
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [30]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir].real
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))

### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
for it in range(ntau):
    response_func[:,it,0] = 1
    if it>=0:#1:
        for ir in range(2):
            response_func[:,it,0] += (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    # elif it==1:
    #     for ir in range(2):
    #         if ir==0:
    #             response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    #             response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    #         else:
    #             response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-radius_theo_map[it].real)
    #             response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-radius_theo_map[it].real)
                
    else:
        for ir in range(1):
            response_func[:,it,0] += (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NE*(mean_lvec[it,ir,0])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)



In [31]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[0]#Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[0]#Ipert[-1]
        
# #### delete the largest 3 and the smallest 3 from response_func_num
# for it in range(ntau):
#     idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>1e2)
#     idxnan = (idxsort) 
#     response_func_num[idxnan,it,0] = np.nan 
    
    
#     idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>1e2)
#     idxnan = (idxsort) 
#     response_func_num[idxnan,it,1] = np.nan 

In [32]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')

ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)
# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([0,3])
ax.set_yticks([0,3])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]])
plt.show()

In [33]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,1],'k',marker='o')
ax.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)

# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('inhibitory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([0,3])
ax.set_yticks([0,3])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]])
plt.show()

In [60]:
np.squeeze(eiglvec_series[ktrial,it,:NE,:2])
print(eigvchn_series[ktrial,it,i])

(0.5566317368018718+0j)


In [90]:
### compute a_m_2^Ea_n_2^E/N  
ktrial = 18
fig,ax = plt.subplots() 
for it in range(ntau-1): 
    lvecE, rvecE = np.zeros((NE,2)),np.zeros((NE,2))
    lvecE= np.squeeze(eiglvec_series[ktrial,it,:NE,:2])
    for i in range(2):
        lvecE[:,i] = lvecE[:,i]*eigvchn_series[ktrial,it,i]
    rvecE= np.squeeze(eigrvec_series[ktrial,it,:NE,:2])
    multiply_EE =rvecE@lvecE.T
    eigv_,_ = np.linalg.eig(multiply_EE)
    ### sort the eigv_  
    eigv_ = np.sort(eigv_.real)[::-1]
    print(eigv_[:10])
    # ax.scatter(eigv_.real,eigv_.imag,c='gray')
    ax.scatter(tau_series[it]*np.ones(1),eigv_[0],c='gray',ls='--')
    # print(eigv_[:5])
    
ax.set_ylim([-2,2])

# multiply_EE =mean_lvec[:,1,0]*mean_rvec[:,1,0]*NI
# multiply_EE +=mean_lvec[:,0,0]*mean_rvec[:,0,0]*NI
# multiply_EE
# ax.plot(tau_series,multiply_EE,c='red',ls='--')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_841596\3173574381.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  lvecE[:,i] = lvecE[:,i]*eigvchn_series[ktrial,it,i]


[8.60696200e-07 4.15152629e-17 4.15152629e-17 3.41212714e-17
 3.41212714e-17 3.28941977e-17 3.28941977e-17 2.58344676e-17
 2.58344676e-17 2.15071567e-17]
[1.09333292e-06 1.11779076e-16 1.11779076e-16 8.01380741e-17
 8.01380741e-17 4.67229130e-17 3.91073389e-17 3.91073389e-17
 3.60753754e-17 3.60753754e-17]
[6.60449103e-17 4.95247079e-17 4.95247079e-17 4.26226204e-17
 4.26226204e-17 3.93319956e-17 3.93319956e-17 3.92626498e-17
 3.89685188e-17 3.89685188e-17]
[9.00335328e-17 9.00335328e-17 7.70289639e-17 7.70289639e-17
 5.77178155e-17 5.77178155e-17 5.37000389e-17 5.09973571e-17
 5.09973571e-17 4.01651953e-17]
[1.10540584e-16 8.68805420e-17 8.24831549e-17 8.24831549e-17
 6.73662938e-17 6.06283053e-17 5.54052933e-17 5.54052933e-17
 4.27265033e-17 4.27265033e-17]
[9.25690925e-17 9.01804608e-17 9.01804608e-17 6.85829759e-17
 6.85829759e-17 6.19931923e-17 6.19931923e-17 5.87502668e-17
 5.87502668e-17 3.55958523e-17]
[5.01386851e-03 5.01386851e-03 2.02066620e-16 1.31428543e-16
 1.05089175e-16

(-2.0, 2.0)

In [ ]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [22]:
params = {'gaverage':gaverage,
          'gamma':gamma,
          'JE':JE,
          'JI':JI,
          'NE':NE,
          'NI':NI,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
       eigrvec_series_rec, eiglvec_series_rec,
        eiglvec0_series, norml0_series, params,
        lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,
        ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
        contributions_lr, contributions_lr_num,
        intg_ov_series,first_perturb_ov,first_perturb_ovP,
        intg_mean_series,intg_std_series,intg_std_num_series,
        intg_crossov_series,intg_crossovPop_series,leig0mean_series,
        norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
       "eigrvec_series_rec, eiglvec_series_rec,"
        "eiglvec0_series, norml0_series, params,"
        "lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,"
        "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
        "contributions_lr, contributions_lr_num,"
        "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
        "intg_mean_series,intg_std_series,intg_std_num_series,"
        "intg_crossov_series,intg_crossovPop_series,leig0mean_series,"
        "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Gaussian_data/DynsStats_Gauss_16April_EXParadoxical_normlr.npz"
# np.savez(data_name, **data)